# 04wk-2: (신경망) – XXXX

최규빈  
2025-03-26

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/lecture.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

In [38]:
11